In [14]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service
import requests
import pandas as pd
import os
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException


path = 'C:/Users/joaop/OneDrive/Documentos/interfaces_web_final_proj/scraping/chromedriver.exe'

In [25]:
# Cria uma instância do Service com o caminho do chromedriver
service = Service(executable_path=path)
# Inicializa o driver passando o serviço
driver = webdriver.Chrome(service=service)

month = 11
year = 2024
day = 1

# urls a fazer scraping
urls = []
for day in range(day, day+1):
    urls.append(f"https://www.basketball-reference.com/friv/dailyleaders.fcgi?month={month}&day={day}&year={year}&type=all")

# Criar uma lista para armazenar os dados
data = []
# Acessando cada URL e realizando o scraping
for url in urls:
    driver.get(url)
    time.sleep(2)  # Espera 2 segundos para a página carregar
    
    # Exemplo de scraping: pegando todos os dados de tabelas ou divs
    try:
        # Localize a tabela ou elemento de interesse na página
        table = driver.find_element(By.XPATH, '//table')  # Ou outro caminho de busca
        
        # Extraindo os dados da tabela
        rows = table.find_elements(By.TAG_NAME, 'tr')
        
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, 'td')
            # Verifique se há colunas na linha
            if len(cols) > 0:
                row_data = [col.text for col in cols]
                # Obter a data diretamente dos parâmetros da URL
                date = f"{year}-{month:02d}-{day:02d}"  # Formato YYYY-MM-DD
                row_data.append(date)
                data.append(row_data)  # Armazena a linha no DataFrame
            print(row_data)
            
    except Exception as e:
        print(f"Erro ao acessar a tabela na URL {url}: {e}")


# Criar um DataFrame com os dados coletados
columns = ["Player", "Tm", "vs", "Opp", "W/L", "MP", "FG", "FGA", "FG%", "3P", "3PA", "3P%", "FT", "FTA", "FT%", "ORB", "DRB", "TRB", "AST", "STL", "BLK", "TOV", "PF", "PTS", "+/-", "GmSc", 'Date']

df = pd.DataFrame(data, columns=columns)

# Mostrar os dados no DataFrame
print(df)

# Finalizando o driver
driver.quit()

['Jordan Walsh', 'BOS', '@', 'CHO', 'W', '6:23', '2', '3', '.667', '2', '3', '.667', '0', '0', '', '0', '0', '0', '0', '0', '0', '0', '0', '6', '+6', '4.7', '2024-11-01']
['Anthony Davis', 'LAL', '@', 'TOR', 'W', '35:43', '14', '20', '.700', '0', '0', '', '10', '11', '.909', '1', '10', '11', '2', '3', '2', '3', '0', '38', '+7', '35.7', '2024-11-01']
['Nikola Vučević', 'CHI', '@', 'BRK', 'L', '33:57', '10', '11', '.909', '3', '4', '.750', '5', '5', '1.000', '3', '8', '11', '3', '1', '1', '0', '0', '28', '-7', '32.6', '2024-11-01']
['Zion Williamson', 'NOP', '', 'IND', 'W', '36:54', '14', '20', '.700', '1', '2', '.500', '5', '8', '.625', '1', '5', '6', '10', '2', '0', '3', '3', '34', '+17', '31.4', '2024-11-01']
['Nikola Jokić', 'DEN', '@', 'MIN', 'L', '39:42', '8', '16', '.500', '2', '3', '.667', '8', '10', '.800', '2', '7', '9', '13', '3', '1', '3', '1', '26', '+6', '30.1', '2024-11-01']
['Aaron Gordon', 'DEN', '@', 'MIN', 'L', '38:03', '11', '18', '.611', '5', '7', '.714', '4', '5', '

In [26]:
df = df[['Player', 'Tm', 'PTS', 'TRB', 'AST', 'Date']]

In [27]:
df

,Player,Tm,PTS,TRB,AST,Date
0,Anthony Davis,LAL,38,11,2,2024-11-01
1,Nikola Vučević,CHI,28,11,3,2024-11-01
2,Zion Williamson,NOP,34,6,10,2024-11-01
3,Nikola Jokić,DEN,26,9,13,2024-11-01
4,Aaron Gordon,DEN,31,11,2,2024-11-01
...,...,...,...,...,...,...
187,Jamal Cain,NOP,0,0,0,2024-11-01
188,Josh Green,CHO,3,1,1,2024-11-01
189,Ariel Hukporti,NYK,0,0,0,2024-11-01
190,Gary Harris,ORL,0,1,0,2024-11-01


In [29]:
df.to_csv('dailyleaders.csv', index=False)